In [80]:
import sys
import os
import warnings
import mne
import torch

warnings.filterwarnings("ignore")
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
mne.set_config("MNE_DATA", os.path.join(os.getcwd(), "data"))
mne.set_config("MNE_DATASETS_BNCI_PATH", os.path.join(os.getcwd(), "data"))
mne.set_config("MNE_DATASETS_EEGBCI_PATH", os.path.join(os.getcwd(), "data"))
mne.set_config("MNE_DATASETS_SHIN_PATH", os.path.join(os.getcwd(), "data"))
mne.set_config("MOABB_RESULTS", os.path.join(os.getcwd(), "results"))
os.makedirs(os.environ["MNE_DATA"], exist_ok=True)
os.makedirs(os.environ["MOABB_RESULTS"], exist_ok=True)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from importlib import reload
from mne.decoding import CSP, PSDEstimator
from sklearn.model_selection import KFold
from torchmetrics.classification import Accuracy

import scripts.transformer.transformer_models as trans
from scripts.dataset.eeg_dataset import EEGDataset
from scripts.features_extract.welch import extract_welch_features
from eeg_logger import logger

import moabb
from moabb.datasets import PhysionetMI
from moabb.paradigms import LeftRightImagery

moabb.set_log_level("info")

In [ ]:
dataset = PhysionetMI(imagined=True, executed=False)
subject_list = np.arange(1, 110, 1)
subject_list = np.delete(subject_list, [87, 91, 99, 103])  # THESE SUBJECTS HAVE INCOMPLETE ANNOTATIONS
dataset.subject_list = subject_list
paradigm = LeftRightImagery(
    channels=[
        "FC5",
        "FC3",
        "FC1",
        "C5",
        "C3",
        "C1",
        "CP5",
        "CP3",
        "CP1",
        "FC2",
        "FC4",
        "FC6",
        "C2",
        "C4",
        "C6",
        "CP2",
        "CP4",
        "CP6",
    ]
)

In [68]:
X_all, y_all, metadata = paradigm.get_data(
    dataset, subjects=dataset.subject_list.tolist(), return_epochs=False
)  # RETURNS DATA FROM RUNS [4, 8, 12]

In [85]:
subjects = metadata["subject"].unique()
X = []
y = []
label_map = {"left_hand": 1, "right_hand": -1}

for idx, subject in enumerate(subjects):

    sess_mask = metadata["subject"] == subject
    X_sess = X_all[sess_mask]
    y_sess = np.array([label_map[label] for label in y_all[sess_mask]])
    # TODO: FEATURE EXTRACT
    X.append(X_sess)
    y.append(y_sess)

X = np.concatenate(X)
y = np.concatenate(y)

# Model training method


In [93]:
def train_model(
    model: torch.nn.Module, train_loader: torch.utils.data.DataLoader, device: torch.device, verbose: bool
) -> None:
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0007, weight_decay=0.0001)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(50):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if verbose:
            logger.info(f"Epoch {epoch+1}/{50}, Loss: {total_loss:.4f}")


def evaluate_model(
    model: torch.nn.Module,
    test_loader: torch.utils.data.DataLoader,
    device: torch.device,
) -> float:
    """
    Computes accuracy of provided model.

    :param model: model to evaluate
    :param test_loader: loader for testing data
    :param device: device to evaluate model on
    """
    acc = Accuracy(task="binary").to(device)
    model.eval()

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output = model(X_batch)
            preds = torch.argmax(output, dim=1)
            acc.update(preds, y_batch)

    return acc.compute().item()


def train(X, y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if device == "cpu":
        logger.warning("Warning - training model on cpu")
    else:
        logger.info("Training model on gpu")

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        train_dataset = EEGDataset(X_train, y_train, cnn_mode=False)
        test_dataset = EEGDataset(X_test, y_test, cnn_mode=False)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

        model = trans.TemporalTransformer(input_size=X_train.shape[1], d_model=64, num_heads=8, num_classes=2)

        logger.info(f"Training {model._get_name()} in fold {fold + 1}...")
        train_model(model, train_loader, device, verbose=False)

        accuracy = evaluate_model(model, test_loader, device)
        logger.info(f"Accuracy for {model._get_name()}  in fold {fold + 1}: {accuracy * 100:.2f}%")
        accuracies.append(accuracy)

    logger.info(f"Mean accuracy across folds: {np.mean(accuracies):.2f}")


train(X, y)

2025-07-23 13:37:34,007 - INFO - Training model on gpu
2025-07-23 13:37:34,124 - INFO - Training TemporalTransformer in fold 1...


IndexError: Target -1 is out of bounds.